### Projeto apresentado à disciplina de Introdução a multimidia, como requisito parcial para a obtenção da aprovação.
#### Autores: Adriano Santana, Bruno, Jefferson Costa, Matheus

#### Importando as dependencias

In [16]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [22]:
df_treino = pd.read_csv('./data/dataset_treino') ### Importando nosso dataframe de treino
df_treino.drop('Unnamed: 0', axis=1, inplace=True) ### Removendo coluna Unnamed

#### Limpando nosso dataframe

In [23]:
sys.path.append('./scripts') ### Importando a função clean text dentro da pasta script
from cleanText import cleanTxt 

###### Removendo @mentions, '#' hash tag, RT, hyperlink 

In [24]:
df_treino['tweet'] = df_treino['tweet'].apply(cleanTxt) ### A coluna tweet recebe agora os tweets limpos
df_treino['tweet'] = df_treino['tweet'].apply(lambda x: x.lower()) ### Colocando todos os tweets em lowerCase

###### Removendo pontuações

In [25]:
sys.path.append('./scripts') ### Importando a função removePunctuation
from removePunctuation import removePunctuation 

In [26]:
df_treino['tweet'] = df_treino.loc[:,'tweet'].apply(removePunctuation) ### Remove pontuações

#### Removendo as stopwords

In [27]:
sys.path.append('./scripts') ### Importando a função removePunctuation
from removeStopWords import removeStopWords 

In [28]:
df_treino['tweet'] = df_treino.loc[:,'tweet'].apply(removeStopWords) ### Remove stopwords com base na língua portuguesa

#### Criando Tokenização

In [29]:
sys.path.append('./scripts') ### Importando a função removePunctuation
from tokenizar import Tokenizar 

In [30]:
tweets_tokenizados = df_treino.loc[:,'tweet'].apply(Tokenizar) ### Criar um array de tokens a partir dos tweets

#### Criando CountVectorizer

Convert a collection of text documents to a matrix of token counts

This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature selection then the number of features will be equal to the vocabulary size found by analyzing the data.

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
tweets_vector = vectorizer.fit_transform(df_treino['tweet'].tolist())

#### Classificação

In [70]:
# Treino de modelo de Machine Learning:
modelo = MultinomialNB()

In [71]:
# Separando dataframe em treino e teste
X_train,X_test,y_train,y_test = train_test_split(tweets_vector, df_treino['Analise'], test_size=0.15)

In [72]:
# Fit
modelo.fit(X_train,y_train)

MultinomialNB()

#### Mostrando resultados

In [73]:
predict = modelo.predict(X_test)

In [74]:
predict

array(['Negativo', 'Negativo', 'Negativo', 'Negativo', 'Neutro',
       'Positivo', 'Neutro', 'Negativo', 'Neutro', 'Negativo', 'Neutro',
       'Negativo', 'Neutro', 'Negativo', 'Neutro', 'Positivo', 'Neutro',
       'Neutro', 'Negativo', 'Positivo', 'Positivo', 'Negativo', 'Neutro',
       'Positivo', 'Neutro', 'Neutro', 'Neutro', 'Positivo', 'Negativo',
       'Negativo', 'Neutro', 'Neutro', 'Negativo', 'Neutro', 'Neutro',
       'Negativo', 'Neutro', 'Neutro', 'Negativo', 'Neutro', 'Neutro',
       'Negativo', 'Neutro', 'Neutro', 'Neutro'], dtype='<U8')

In [75]:
print(classification_report(modelo.predict(X_test),y_test))

              precision    recall  f1-score   support

    Negativo       0.80      0.75      0.77        16
      Neutro       0.83      0.87      0.85        23
    Positivo       0.50      0.50      0.50         6

    accuracy                           0.78        45
   macro avg       0.71      0.71      0.71        45
weighted avg       0.78      0.78      0.78        45



### Classificando dataframe primeiro turno

In [76]:
df_primeiro_turno = pd.read_csv('./data/df_primeiro_turno_original') # Lendo CSV

In [77]:
tweets_original = df_primeiro_turno[['tweet','palavra_chave']]

In [78]:
df_primeiro_turno.drop('Unnamed: 0', axis=1, inplace=True) # Removida coluna unnamed

In [79]:
df_primeiro_turno['tweet'] = df_primeiro_turno['tweet'].apply(cleanTxt) ### A coluna tweet recebe agora os tweets limpos
df_primeiro_turno['tweet'] = df_primeiro_turno['tweet'].apply(lambda x: x.lower()) ### Colocando todos os tweets em lowerCase

In [80]:
df_primeiro_turno['tweet'] = df_primeiro_turno.loc[:,'tweet'].apply(removePunctuation) ### Remove pontuações

In [81]:
df_primeiro_turno['tweet'] = df_primeiro_turno.loc[:,'tweet'].apply(removeStopWords) ### Remove stopwords com base na língua portuguesa

In [82]:
tweets_primeiro_turno = vectorizer.transform(df_primeiro_turno['tweet'].tolist())

In [83]:
tweets_primeiro_turno

<2000x1006 sparse matrix of type '<class 'numpy.int64'>'
	with 23435 stored elements in Compressed Sparse Row format>

In [86]:
classificacao_primeiro_turno = modelo.predict(tweets_primeiro_turno)

In [87]:
classificacao_primeiro_turno

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Neutro', 'Neutro'],
      dtype='<U8')

In [19]:
from estatisticas import porcent, moda
moda(predict) #Just to test with predict, adjust to use with each candidate
porcent(predict, "Positivo") 
porcent(predict, "Neutro") 
porcent(predict, "Negativo") 

Moda: Neutro
Positivos: 26.67 %
Neutros: 51.11 %
Negativos: 22.22 %
